In [1]:
# wikipedia stuff
import os, sys, datetime, re
import numpy as np
import tensorflow as tf
import random
import ag.logging as log
from tqdm import tqdm_notebook, tnrange
from tqdm import tqdm
buf="$" * 40
log.set(5)

In [2]:
# open dataset
class dataset(): pass
dataset.text = open('/pub/dataset/wiki/wiki.test.raw', encoding='utf-8', errors="surrogateescape").read()
dataset.len = len(dataset.text)
print("# Number of Staring Characters: {}".format(dataset.len))
# oh NOOOO!!!! we have to trunkate or dataset in a weird place... this could hoze our results!
dataset.text = dataset.text[:int(dataset.len/4)] # gotta cut it by 80%
dataset.len = len(dataset.text)
print("# Number of Working Characters: {}".format(dataset.len))
dataset.sample = dataset.text[:100]
dataset.chars = sorted(list(set(dataset.text)))
dataset.len_chars = len(dataset.chars)

# Number of Staring Characters: 1288556
# Number of Working Characters: 322139


In [3]:
print("# Number of Character: {}".format(dataset.len))
print("# Number of Unique Characters: {}".format(dataset.len_chars))
print("# Sample: {}\n".format(dataset.sample))
print("# All Unique Characters:\n {}".format(dataset.chars))

# Number of Character: 322139
# Number of Unique Characters: 155
# Sample:  
 = Robert Boulter = 
 
 Robert Boulter is an English film , television and theatre actor . He had 

# All Unique Characters:
 ['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '©', '°', 'É', 'à', 'á', 'ã', 'ä', 'æ', 'è', 'é', 'ê', 'í', 'ñ', 'ó', 'ö', 'ü', 'ě', 'ī', 'Ō', 'ō', 'ū', 'ǐ', 'ǜ', 'ə', 'έ', 'δ', 'ε', 'ι', 'λ', 'μ', 'ν', 'ο', 'π', 'ς', 'σ', 'τ', 'υ', 'ό', '‑', '–', '—', '’', '−', '♯', '伊', '傳', '八', '勢', '史', '型', '士', '大', '律', '成', '戦', '春', '望', '杜', '東', '甫', '甲', '聖', '艦', '處', '衛', '解', '詩', '贈', '邵', '鉄', '集']


In [4]:
char2id = dict((c, i) for i, c in enumerate(dataset.chars))
id2char = dict((i, c) for i, c in enumerate(dataset.chars))

In [5]:
# probibilty of each next character
def sample(pred):
    r = random.uniform(0,1)
    s= 0
    l = len(pred)
    char_id =  l-1 
    for i in range(l):
        s += pred[i]
        if s >= r:
            char_id = i
            break
    
    one_hot_char = np.zeros(shape=[dataset.len_chars])
    one_hot_char[char_id] = 1.0
    return one_hot_char

In [6]:
# batch and options
class options(): pass
options.batch_section = 50
options.skip = 2

In [7]:
# do some work... create the sections list
dataset.sections = []
dataset.next_chars = []
for i in range(0, dataset.len - options.batch_section, options.skip):
    dataset.sections.append(dataset.text[i: i + options.batch_section])
    dataset.next_chars.append(dataset.text[i + options.batch_section])
dataset.len_sections = len(dataset.sections)
print("Length of Section: {}".format(dataset.len_sections))

Length of Section: 161045


In [8]:
# if you want to explore your memory size... go for /3
# x = np.zeros((int(644253/4), int(50), 159))

In [9]:
# Create input data sctructure placeholders
tensor_inputs = np.zeros((dataset.len_sections, options.batch_section, dataset.len_chars))
tensor_labels = np.zeros((dataset.len_sections, dataset.len_chars))

In [10]:
# for each_position and each batch of 50, one at a time, in all batches with index.
for i, section in enumerate(dataset.sections):
    # for this poistion and this character in this section with its index
    for j, char in enumerate(section):
        # place a datapoint at for T at poistion 4000 h is the next char
        tensor_inputs[i,j,char2id[char]] = 1
    tensor_labels[i, char2id[dataset.next_chars[i]]] = 1
print("Len of labels:\t{}\nLabels: {}".format(len(tensor_labels),tensor_labels))

Len of labels:	161045
Labels: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [27]:
#### BOOMSKI here we go... data wash complete.
options.batch_size = 512
options.iters = 1e4
options.log_every = 50
options.save_every = 100
options.hidden_nodes = 1024
options.max_text = 144

In [12]:
starting_text = "The world is filled with music."
if True:
    save_path = "/pub/dataset/wiki/log/perm"
    if os.path.isdir(save_path):
        pass
    else:
        "had to make the dir."
        tf.gfile.MakeDirs(save_path)
else:    
    save_path = "/home/eric/datasets/wiki/log"
    if os.path.isdir(save_path):
        print("check")
        all_logs = os.listdir(save_path)
        num_logs = len(all_logs)
        new_path = os.path.join(save_path,"wikigen_{}".format(num_logs))
        tf.gfile.MakeDirs(new_path)
        if os.path.isdir(new_path):
            save_path = new_path
        else:
            print("error creating folder")

    else:
        print("errors finding path")
print("Save path is: {}".format(save_path))
save_filename = "ckpt"

Save path is: /pub/dataset/wiki/log/perm


In [13]:
print("training data size: {}".format(len(tensor_inputs)))
print("steps per epoch: {}".format(int(len(tensor_inputs)/options.batch_size)))

training data size: 161045
steps per epoch: 314


In [14]:
with tf.variable_scope("inputs"):
    global_step = tf.Variable(0, name='global_step', trainable=False)
    # global_step = tf.Variable('global_step',trainable=False)
    learn_rate = tf.train.exponential_decay(
            0.1, global_step,
            1e-7, 0.87, staircase=True,
            name="Learn_decay")

    #tf.add_to_collection("learn_rate", learn_rate)
    #tf.summary.scalar("learn_rate", learn_rate)
    # input placeholders still inside of inputs scope
    input_holder = tf.placeholder(tf.float32, 
                                  [options.batch_size,
                                   options.batch_section,
                                   dataset.len_chars],
                                  name="Input_Tensor")

    input_label = tf.placeholder(tf.float32, 
                                 [options.batch_size,
                                  dataset.len_chars],
                                 name="Input_label")
#tf.add_to_collection("input_tensor", input_holder)
#tf.add_to_collection("input_label", input_label)
# input game, out game, forget gate, state

In [15]:
with tf.variable_scope("gates"):
    #'''input gate'''
    # weights for input
    w_ii = tf.Variable(tf.truncated_normal([dataset.len_chars,
                                           options.hidden_nodes],
                                           -0.1,0.1),
                       name="in_weights")
    # weights for previous output
    w_io = tf.Variable(tf.truncated_normal([options.hidden_nodes,
                                            options.hidden_nodes],
                                           -0.1,0.1),
                       name="previous_weights")
    # bias
    b_i = tf.Variable(tf.zeros([1, options.hidden_nodes]),
                      name="input_bias")
    # for re optimizing
    #tf.add_to_collection("in_weights", w_ii)
    #tf.add_to_collection("previous_weights", w_io)
    #tf.add_to_collection("input_bias", b_i)

    #'''forget gate'''
    # weights for input
    w_fi = tf.Variable(tf.truncated_normal([dataset.len_chars,
                                           options.hidden_nodes],
                                           -0.1,0.1),
                       name="forget_weights")
    # weights for previous output
    w_fo = tf.Variable(tf.truncated_normal([options.hidden_nodes,
                                            options.hidden_nodes],
                                           -0.1,0.1),
                       name="forget_previous_weights")
    # bias
    b_f = tf.Variable(tf.zeros([1, options.hidden_nodes]),
                      name="forget_bias")
    # for re optimizing
    #tf.add_to_collection("forget_weights", w_fi)
    #tf.add_to_collection("forget_previous_weights", w_fo)
    #tf.add_to_collection("forget_bias", b_f)

    #'''output gate'''
    # weights for input
    w_oi = tf.Variable(tf.truncated_normal([dataset.len_chars,
                                           options.hidden_nodes],
                                           -0.1,0.1),
                       name="out_weights")
    # weights for previous output
    w_oo = tf.Variable(tf.truncated_normal([options.hidden_nodes,
                                            options.hidden_nodes],
                                           -0.1,0.1),
                       name="out_previous_weights")
    # bias
    b_o = tf.Variable(tf.zeros([1, options.hidden_nodes]),
                      name="output_bias")
    # for re optimizing
    #tf.add_to_collection("output_weights", w_oi)
    #tf.add_to_collection("out_previous_weights", w_oo)
    #tf.add_to_collection("output_bias", b_o)

    #'''memory gate'''
    # weights for input
    w_ci = tf.Variable(tf.truncated_normal([dataset.len_chars,
                                           options.hidden_nodes],
                                           -0.1,0.1),
                       name="memory_weights")
    # weights for previous output
    w_co = tf.Variable(tf.truncated_normal([options.hidden_nodes,
                                            options.hidden_nodes],
                                           -0.1,0.1),
                       name="memory_previous_weights")
    # bias
    b_c = tf.Variable(tf.zeros([1, options.hidden_nodes]),
                      name="memory_bias")
    # for re optimizing
    #tf.add_to_collection("memory_weights", w_ci)
    #tf.add_to_collection("memory_previous_weights", w_co)
    #tf.add_to_collection("memory_bias", b_c)

In [16]:
def lstm_cell(i, o, state):
    input_gate = tf.sigmoid(tf.matmul(i, w_ii) + tf.matmul(o, w_io) + b_i)
    #(input * forget weights) + (output * weights for previous output) + bias
    forget_gate = tf.sigmoid(tf.matmul(i, w_fi) + tf.matmul(o, w_fo) + b_f)
    #(input * output weights) + (output * weights for previous output) + bias
    output_gate = tf.sigmoid(tf.matmul(i, w_oi) + tf.matmul(o, w_oo) + b_o)
    #(input * internal state weights) + (output * weights for previous output) + bias
    memory_cell = tf.sigmoid(tf.matmul(i, w_ci) + tf.matmul(o, w_co) + b_c)

    #...now! multiply forget gate * given state    +  input gate * hidden state
    state = forget_gate * state + input_gate * memory_cell
    #squash that state with tanh nonlin (Computes hyperbolic tangent of x element-wise)
    #multiply by output
    output = output_gate * tf.tanh(state)
    return output, state

In [17]:
### OHHHH WE ARE GETTING CLOSE! ###
# """ Initialize the session """
# init_op = tf.global_variables_initializer
# 
# """create summary op"""
# merged = tf.summary.merge_all
# 
# """create saver object"""
# saver = tf.train.Saver(
#             #var_list={"{}".format(v): v for v in [tf.model_variables()]},
#             write_version=tf.train.SaverDef.V2,
#             sharded=True,
#             keep_checkpoint_every_n_hours=.001
#             )

In [18]:
output = tf.zeros([options.batch_size, options.hidden_nodes])
state = tf.zeros([options.batch_size, options.hidden_nodes])
print(output, state)

Tensor("zeros:0", shape=(512, 1024), dtype=float32) Tensor("zeros_1:0", shape=(512, 1024), dtype=float32)


In [19]:
###... still chopping wood...
outputs_all_i = []
labels_all_i = []
for i in range(options.batch_section):
    #calculate state and output from LSTM
    output, state = lstm_cell(input_holder[:, i, :], output, state)
    #to start, 
    if i == 0:
        print("working first set")
        #store initial output and labels
        outputs_all_i = output
        labels_all_i = input_holder[:, i+1, :]
    #for each new set, concat outputs and labels
    if i != options.batch_section - 1:
        # print("working set {}".format(i+1))
        #concatenates (combines) vectors along a dimension axis, not multiply
        outputs_all_i = tf.concat([outputs_all_i, output], 0)
        labels_all_i = tf.concat([labels_all_i, input_holder[:, i+1, :]], 0)
    else:
        #final store
        print("working final set")
        outputs_all_i = tf.concat([outputs_all_i, output], 0)
        labels_all_i = tf.concat([labels_all_i, input_label], 0)
print("process complete")

working first set
working final set
process complete


In [20]:
# Classificator
with tf.variable_scope("superNet_layer"):
    sw = tf.Variable(tf.truncated_normal([options.hidden_nodes, dataset.len_chars], -0.1,0.1),
                    name="superNet_weights")
    sb = tf.Variable(tf.zeros(dataset.len_chars),name="superNet_biases")
tf.summary.histogram('superNet_weights', sw); 
tf.summary.histogram('superNet_biases', sb);

In [21]:
# with tf.device("gpu0"):
with tf.variable_scope("training_op"):
    # google like the word logits.. it represents the "final Layer"
    final_layer = tf.matmul(outputs_all_i, sw) + sb
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels_all_i, logits=final_layer))
    tf.add_to_collection("softmax_activations", cost);
    tf.summary.scalar("softmax_activations", cost);
    tf.summary.histogram('softmax_activations', cost);
    # should have them all here lined up... but im not there yet...
    train_op = tf.train.AdagradOptimizer(learn_rate).minimize(cost, global_step=global_step)
    train_op = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)
    tf.add_to_collection("train_op", train_op);

In [22]:
test_data = tf.placeholder(tf.float32, shape=[1, dataset.len_chars])
test_output = tf.Variable(tf.zeros([1, options.hidden_nodes]))
test_state = tf.Variable(tf.zeros([1, options.hidden_nodes]))

#Reset at the beginning of each test
reset_test_state = tf.group(test_output.assign(tf.zeros([1, options.hidden_nodes])), 
                            test_state.assign(tf.zeros([1, options.hidden_nodes])))

#LSTM
test_output, test_state = lstm_cell(test_data, test_output, test_state)
with tf.variable_scope("test_op"):
    test_op = tf.nn.softmax(tf.matmul(test_output, sw) + sb)
    tf.add_to_collection("test_softmax_activations", test_op);
    tf.summary.scalar("test_softmax_activations", test_op);
    tf.summary.histogram('test_softmax_activations', test_op);

In [23]:
""" Initialize the session """
init_op = tf.global_variables_initializer

"""create summary op"""
merged = tf.summary.merge_all()
print("merged: {}".format(merged))

"""create saver object"""
saver = tf.train.Saver()

merged: Tensor("Merge/MergeSummary:0", shape=(), dtype=string)


In [24]:
log.info("Starting the wikigen Training!")
#with tf.Graph().as_default():
try:
    if sess:
        log.debug("closing old sess")
        sess.close()
except: pass
sess = tf.InteractiveSession()
log.debug("session started")
#init graph, load model
sess.run(init_op())
train_writer = tf.summary.FileWriter(save_path, sess.graph)
# test_writer = tf.summary.FileWriter(self.options.logDir, sess.graph)
log.debug("init started")
offset = 0
iters = 1  # testing
log.info("Training for {} iters".format(iters))

 *I* Starting the wikigen Training!
 ~D~ session started
 ~D~ init started
 *I* Training for 1 iters


In [ ]:
x = options.iters

msg = "Starting the Training for {} iterations".format(x)
print(msg)
for step in tqdm(range(int(x))):
    global_step += 1
    offset = offset % len(tensor_inputs)
    if offset <= (len(tensor_inputs) - options.batch_size):
        batch_data = tensor_inputs[offset: offset + options.batch_size]
        batch_labels = tensor_labels[offset: offset + options.batch_size]
        offset += options.batch_size
    else:
        to_add = options.batch_size - (len(tensor_inputs) - offset)
        batch_data = np.concatenate((tensor_inputs[offset: len(tensor_inputs)], tensor_inputs[0: to_add]))
        batch_labels = np.concatenate((tensor_labels[offset: len(tensor_inputs)], tensor_labels[0: to_add]))
        offset = to_add
    # this is finally it ... GEEZEEZEZ
    feed_dict={input_holder: batch_data, input_label: batch_labels}
    _, loss_, global_step_, learn  = sess.run([train_op,cost, global_step, learn_rate], feed_dict=feed_dict)
    
    if step % options.log_every == 0:        
        msg = "{} :: Current Step: {}\n\t-Loss: {}\n\t-Learn rate: {}\n{}\n".format(datetime.time(), 
                                                                                    global_step_, 
                                                                                    cost, 
                                                                                    learn, 
                                                                                    buf)
        
        if step % options.save_every == 0:
            reset_test_state.run()
            test_generated = starting_text
            # log.debug("in a testing phase")
            for i in tqdm(range(len(starting_text) - 1)):
                test_X = np.zeros((1, dataset.len_chars))
                test_X[0, char2id[starting_text[i]]] = 1.
                test_feed={test_data: test_X}
                _ = sess.run(test_op, feed_dict=test_feed)
            # test generated
            test_X = np.zeros((1, dataset.len_chars))
            test_X[0, char2id[starting_text[-1]]] = 1.
            # creating test
            for i in tqdm(range(options.max_text)):
                prediction = test_op.eval({test_data: test_X})[0]
                next_char_one_hot = sample(prediction)
                next_char = id2char[np.argmax(next_char_one_hot)]
                test_generated += next_char
                test_X = next_char_one_hot.reshape((1, dataset.len_chars))

            #print(buf)
            # print(test_generated)
            with open(os.path.join(save_path, "generated.txt"), "a") as file:
                file.write("\n{0} Start Test {0}\n".format(buf))
                file.write(test_generated)
                file.write("\n{0} End of Test {0}\n".format(buf))
                            
            saver.save(sess, save_path + '/model', global_step=global_step_)
print(msg)




  0%|          | 0/10000 [00:00<?, ?it/s]

Starting the Training for 10000.0 iterations







  0%|          | 0/30 [00:00<?, ?it/s]


100%|██████████| 30/30 [00:00<00:00, 458.15it/s]


  0%|          | 0/144 [00:00<?, ?it/s]


 31%|███       | 44/144 [00:00<00:00, 439.49it/s]


 61%|██████    | 88/144 [00:00<00:00, 438.84it/s]


 94%|█████████▍| 136/144 [00:00<00:00, 443.16it/s]


100%|██████████| 144/144 [00:00<00:00, 439.02it/s]

  0%|          | 1/10000 [00:11<32:10:15, 11.58s/it]

  0%|          | 2/10000 [00:21<30:57:25, 11.15s/it]

  0%|          | 3/10000 [00:32<30:23:00, 10.94s/it]

  0%|          | 4/10000 [00:41<29:23:56, 10.59s/it]

  0%|          | 5/10000 [00:51<28:35:08, 10.30s/it]

  0%|          | 6/10000 [01:01<28:12:54, 10.16s/it]

  0%|          | 7/10000 [01:11<27:53:31, 10.05s/it]

  0%|          | 8/10000 [01:20<27:30:59,  9.91s/it]

  0%|          | 9/10000 [01:30<27:14:17,  9.81s/it]

  0%|          | 10/10000 [01:40<27:12:13,  9.80s/it]

  0%|          | 11/10000 [01:49<27:03:42,  9.75s/it]

  0%|          | 12/10000 [01:59<26:57:50,  9.72s/it]

 

In [ ]:
import tensorflow as tf
import os
import ag.logging as log
save_path = '/pub/dataset/wiki/log/perm'

sess = tf.Session()
# new_saver = tf.train.import_meta_graph(os.path.join(save_path, 'model-24.meta'))
# starting_text
#init graph, load model
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    model = tf.train.latest_checkpoint(save_path)
    
    saver = tf.train.Saver()
    saver.restore(sess, model)

In [ ]:
sess.run(tf.global_variables_initializer())
# saver = tf.train.Saver()
log.info("Initilized Session")
# tf.global_variables_initializer().run()
model = tf.train.latest_checkpoint(save_path)
saver.restore(sess, model)
log.info("Restored most current training")
# set input variable to generate chars from
sess.run(reset_test_state)
test_generated = starting_text
log.info("Loading Starting text {}.".format(starting_text))
#for every char in the input sentennce
for i in range(len(starting_text) - 1):
    #initialize an empty char store
    test_X = np.zeros((1, dataset.len_chars))
    #store it in id from
    test_X[0, char2id[starting_text[i]]] = 1.
    #feed it to model, test_prediction is the output value
    _ = sess.run(test_op, feed_dict={test_data: test_X})
log.info("Test Run.")

#where we store encoded char predictions
test_X = np.zeros((1, dataset.len_chars))
test_X[0, char2id[starting_text[-1]]] = 1.

In [ ]:
#lets generate 500 characters
for i in range(500):
    #get each prediction probability
    prediction = test_op.eval({test_data: test_X})[0]
    #one hot encode it
    next_char_one_hot = sample(prediction)
    #get the indices of the max values (highest probability)  and convert to char
    next_char = id2char[np.argmax(next_char_one_hot)]
    #add each char to the output text iteratively
    test_generated += next_char
    #update the 
    test_X = next_char_one_hot.reshape((1, dataset.len_chars))

print(test_generated)